#Scope
The scope of this code includes:

1. Authenticating and initializing the GEE API.
2. Retrieving and processing satellite imagery data - SAR(Sentinel-1)
3. Merging various environmental factors and ASI(maritime)
5. Exporting the results to Google Drive.

#Introduction

* This Python script helps to analyze SAR for oil spill over time using Google Earth Engine (GEE). The results are exported to Google Drive in CSV format for further analysis.

* The code has been converted from Google Earth Engine JavaScript to Python using the **geemap** package, which simplifies the process of working with GEE in Python and enables easy conversion of scripts from JavaScript to Python.


In [ ]:

#!pip install geemap
#!pip install earthengine-api

In [ ]:
#Import required libraries
import ee
import pandas as pd


ee.Authenticate()
ee.Initialize(project='ee-pranalitalla8')


In [ ]:
#import os
#os.environ['EE_PROJECT_ID'] = 'ee-pranalitalla8'
#ee.Initialize(project=os.getenv('EE_PROJECT_ID'))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import geemap
from datetime import datetime

## Geemap
Geemap is a Python package for interactive geospatial analysis and visualization with Google Earth Engine (GEE), helps for existing GEE users who would like to transition from the GEE JavaScript API to Python API. The automated JavaScript-to-Python conversion module of the geemap package can greatly reduce the time needed to convert existing GEE JavaScripts to Python scripts and Jupyter notebooks.

[geemap.org](https://geemap.org)

In [ ]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

### Variables and collections
 Various collections and images required for our analysis in Google Earth Engine (GEE). These collections include SAR (Sentinel-1- C band), JRC Water History.

In [ ]:
ROI = ee.Geometry.Rectangle([-77.3, 36.8, -75, 39.7])
start_date = ee.Date('2015-01-01')
end_date = ee.Date('2025-12-31')

### Get Available SAR Dates

In [ ]:
def get_available_sar_dates(roi, start, end):
    #Get all available SAR dates for the ROI and time range
    sarCol_all = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

    # Get unique dates
    dates = sarCol_all.aggregate_array('system:time_start') \
        .map(lambda x: ee.Date(x).format('YYYY-MM-dd')) \
        .distinct() \
        .sort()

    return dates.getInfo()

### Single-Date Processing Function

In [ ]:
def process_sar_date(date_str, roi):
    #Process SAR data for a single date
    try:
        date = ee.Date(date_str)

        # SAR Setup
        sarCol = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filterBounds(roi) \
            .filterDate(date, date.advance(1,'day')) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

        # Check if any images exist
        if sarCol.size().getInfo() == 0:
            print(f'No SAR images for {date_str}')
            return None

        # Single SAR composite
        sar = sarCol.median().clip(roi)
        vhvv = sar.select('VH').divide(sar.select('VV')).rename('VH_VV_ratio')

        desc = sarCol.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).size()
        asc = sarCol.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).size()
        orbitNum = ee.Algorithms.If(desc.gt(asc), 1, 0)
        orbitName = ee.String(ee.Algorithms.If(desc.gt(asc), 'DESC', 'ASC'))
        orbitDirBand = ee.Image.constant(orbitNum).clip(roi).rename('orbit_direction')

        # JRC Water Mask
        jrcWater = ee.ImageCollection('JRC/GSW1_4/YearlyHistory') \
            .filterDate('2021-01-01', '2021-12-31') \
            .select('waterClass') \
            .mosaic() \
            .clip(roi) \
            .gte(3)

        # Masks
        oilMask = sar.select('VV').lt(-22).And(jrcWater)
        nonOilWaterMask = sar.select('VV').gte(-22).And(sar.select('VV').gt(-18)).And(jrcWater)

        # Stack features
        allFeatures = sar.select(['VV', 'VH', 'angle']) \
            .addBands(vhvv) \
            .addBands(orbitDirBand)

        # Labeling function
        def addBasicCols(feature, label):
            coords = feature.geometry().coordinates()
            properties = {
                'oil_candidate': label,
                'date': date.format('YYYY-MM-dd'),
                'latitude': ee.Number(coords.get(1)),
                'longitude': ee.Number(coords.get(0)),
                'orbit_direction': orbitNum,
                'orbit_type': orbitName,
                'VV': feature.get('VV'),
                'VH': feature.get('VH'),
                'angle': feature.get('angle'),
                'VH_VV_ratio': feature.get('VH_VV_ratio')
            }
            return ee.Feature(feature.geometry(), properties)

        # Sample points
        sampledOilPoints = allFeatures \
            .updateMask(oilMask) \
            .sample(region=roi, scale=30, numPixels=500, seed=101, geometries=True) \
            .map(lambda f: addBasicCols(f, 1))

        sampledWaterPoints = allFeatures \
            .updateMask(nonOilWaterMask) \
            .sample(region=roi, scale=30, numPixels=500, seed=201, geometries=True) \
            .map(lambda f: addBasicCols(f, 0))

        # Combine
        allLabeledPoints = sampledOilPoints.merge(sampledWaterPoints)

        return allLabeledPoints

    except Exception as e:
        print(f'Error processing {date_str}: {e}')
        return None

### Automated Loop Processing

In [ ]:
print("Getting available SAR dates...")
available_dates = get_available_sar_dates(ROI, start_date, end_date)
print(f"Found {len(available_dates)} available SAR dates")
print("First 10 dates:", available_dates[:10])

Getting available SAR dates...
Found 549 available SAR dates
First 10 dates: ['2015-03-10', '2015-03-17', '2015-08-25', '2015-09-18', '2016-04-09', '2016-04-16', '2016-07-02', '2016-07-09', '2016-09-24', '2016-10-01']


In [ ]:
#Process multiple dates (adjust range as needed)
#dates_to_process = available_dates[:20]  # Process first 20 dates
dates_to_process = available_dates
all_results = []
successful_dates = []

for i, date_str in enumerate(dates_to_process):
    print(f'Processing {i+1}/{len(dates_to_process)}: {date_str}...')

    result = process_sar_date(date_str, ROI)

    if result is not None:
        all_results.append(result)
        successful_dates.append(date_str)
        print(f'{date_str} completed')
    else:
        print(f'{date_str} skipped')

print(f"\nSuccessfully processed {len(successful_dates)} dates")


Processing 1/549: 2015-03-10...
2015-03-10 completed
Processing 2/549: 2015-03-17...
2015-03-17 completed
Processing 3/549: 2015-08-25...
2015-08-25 completed
Processing 4/549: 2015-09-18...
2015-09-18 completed
Processing 5/549: 2016-04-09...
2016-04-09 completed
Processing 6/549: 2016-04-16...
2016-04-16 completed
Processing 7/549: 2016-07-02...
2016-07-02 completed
Processing 8/549: 2016-07-09...
2016-07-09 completed
Processing 9/549: 2016-09-24...
2016-09-24 completed
Processing 10/549: 2016-10-01...
2016-10-01 completed
Processing 11/549: 2016-10-06...
2016-10-06 completed
Processing 12/549: 2016-10-08...
2016-10-08 completed
Processing 13/549: 2016-10-13...
2016-10-13 completed
Processing 14/549: 2016-10-18...
2016-10-18 completed
Processing 15/549: 2016-10-25...
2016-10-25 completed
Processing 16/549: 2016-10-30...
2016-10-30 completed
Processing 17/549: 2016-11-06...
2016-11-06 completed
Processing 18/549: 2016-11-11...
2016-11-11 completed
Processing 19/549: 2016-11-18...
2016

### Export Combined Results

In [ ]:
if all_results:
    print("Combining all results (server-side)...")

    # Combine on the server to avoid recursion
    combined_collection = ee.FeatureCollection(all_results).flatten()

    # Export
    task = ee.batch.Export.table.toDrive(
        collection=combined_collection,
        description=f'Chesapeake_SAR_Multi_Date_{len(successful_dates)}_dates',
        fileFormat='CSV'
    )

    task.start()
    print(f'Export started! Processing {len(successful_dates)} dates.')
else:
    print('No results to export.')


Combining all results (server-side)...
Export started! Processing 549 dates.


In [ ]:
#Print summary
print(f"\nSUMMARY:")
print(f"Total available dates: {len(available_dates)}")
print(f"Dates processed: {len(dates_to_process)}")
print(f"Successful: {len(successful_dates)}")
print(f"Failed: {len(dates_to_process) - len(successful_dates)}")


SUMMARY:
Total available dates: 549
Dates processed: 549
Successful: 549
Failed: 0


Time Period, Data Collections, Variables
To conduct our analysis, we begin by defining the time period and identifying the necessary data collections. We specify the start and end dates for the analysis, set up relevant feature collections, and initialize variables essential for soil erosion analysis. This includes:

* Time Period: Establishing the time frame using start and end dates to define the duration of the analysis.
* Data Collections: Selecting and defining  collections such as validated reservoirs and catchments that are crucial for our study.
* Variables: Setting up variables to manage VV,VH, errors, and date lists for time series analysis.

These initial setups enable us to retrieve and process satellite imagery, compute various environmental factors.